In [ ]:
import numpy as np
import birdwatcher as bw
import birdwatcher.movementdetection as md

from birdwatcher.plotting import imshow_frame
import matplotlib.pyplot as plt
%matplotlib inline

This notebook shows movement detection the easy way, using a high-level function.

In [ ]:
vfs = bw.testvideosmall() # get a videofilestream object

# Example with MOG2 algorithm

One high-level function can detect movement based on background subtraction. It returns three arrays, which are disk-based darr arrays. They can be very large and hence not fit in RAM. 'coords' provided the coordinates of detected pixels in a ragged array. 'coordscount' the number of pixels per frame, 'coordsmean' the spatial average.

In [ ]:
coords, coordscount, coordsmean = md.detect_movementmog2(vfs, morphologyex=2, analysispath='output/',
                                                      ignore_firstnframes=50, overwrite=True,
                                                      VarThreshold=70, NMixtures=8, History=3)

Let's look at frame 200

In [ ]:
coords[200]

we can view the results as a black and white image by the `get_frame` method, which returns a frame instead of coordinates.

In [ ]:
frame = coords.get_frame(130)
imshow_frame(frame)

let's look at the original frame

In [ ]:
imshow_frame(vfs.get_frame(130))

we can create a video of the results as well

In [ ]:
coords.tovideo('output/zf20s_coords.mp4', framerate=vfs.avgframerate)

or a selection of the results by indicating start and end frame numbers

In [ ]:
coords.tovideo('output/zf20s_coords_selection.mp4', startframe=100, endframe=200, framerate=vfs.avgframerate)

if you want to do more things before saving to video, just use `iter_frames` which turns it into a Frames object with many more methods. Make sure you use three color channels and set coordinates to value 255 if you want them white.

In [ ]:
frames = coords.iter_frames(startframe=100, endframe=200, nchannels=3, value=255)
_ = frames.draw_framenumbers().tovideo('output/zf20s_coords_selection_framenumbers.mp4', framerate=vfs.avgframerate)

or we can look at the data in plots

In [ ]:
plt.plot(coordscount)
plt.title('number of pixels above treshold')
plt.xlabel('framenumber')
plt.ylabel('number of pixels')

In [ ]:
plt.plot(coordsmean)
plt.title('coordinates of pixels above treshold')
plt.xlabel('framenumber')
plt.ylabel('pixel')
plt.legend(['left-right', 'top-bottom'])

let's make a video with the original frames + the mean coordinate superimposed on it as a circle

In [ ]:
vfs_circle = vfs.iter_frames().draw_framenumbers().draw_circles(coordsmean).tovideo('output/zf20s_coords_center.mp4', framerate=vfs.avgframerate)

In [ ]:
frame = coords.get_frame(130)
imshow_frame(frame)

There is also a high-level function that does this for you, and better ;-) It will produce a video next to the original one with results superimposed.

In [ ]:
vfs_results = md.create_movementvideo(vfs, coords, videofilepath='output/movementvideoexample.mp4')

In [ ]:
imshow_frame(vfs_results.get_frame(130))

In [ ]:
coords.iter_frames?

# A different algorithm: KNN

In [ ]:
coords, coordscount, coordsmean = md.detect_movementknn(vfs, morphologyex=2, analysispath='output/',
                                                     ignore_firstnframes=50, overwrite=True,
                                                     kNNSamples=0, NSamples=6, Dist2Threshold=500, 
                                                     DetectShadows=False)

In [ ]:
frame = coords.get_frame(130)
imshow_frame(frame)

# Yet another algorithm, LSBP

In [ ]:
coords, coordscount, coordsmean = md.detect_movementlsbp(vfs, morphologyex=2, analysispath='output/',
                                                     ignore_firstnframes=50, overwrite=True,
                                                     mc=0, nSamples=20, LSBPRadius=16)

In [ ]:
frame = coords.get_frame(130)
imshow_frame(frame)

In [ ]:
coords.tovideo('output/zf20s_coords_lsbp.mp4', framerate=vfs.avgframerate)